In [1]:
import torch
from sentence_transformers import SentenceTransformer
model_id = "sentence-transformers/msmarco-distilbert-base-tas-b"
model = SentenceTransformer(model_id)


In [2]:
model.tokenizer

DistilBertTokenizerFast(name_or_path='/Users/latchari/.cache/torch/sentence_transformers/sentence-transformers_msmarco-distilbert-base-tas-b/', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [4]:
model.tokenizer.padding_side

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 model.tokenizer.padding                                                                      │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DistilBertTokenizerFast' object has no attribute 'padding'

In [5]:
model_id_2 = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
model_2 = SentenceTransformer(model_id_2)

In [7]:
model_2.tokenizer

BertTokenizerFast(name_or_path='/Users/latchari/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [31]:
model_id_3 = "sentence-transformers/paraphrase-MiniLM-L3-v2"
model_3 = SentenceTransformer(model_id_2)

In [32]:
help(model_3)

Help on SentenceTransformer in module sentence_transformers.SentenceTransformer object:

class SentenceTransformer(torch.nn.modules.container.Sequential)
 |  SentenceTransformer(model_name_or_path: Optional[str] = None, modules: Optional[Iterable[torch.nn.modules.module.Module]] = None, device: Optional[str] = None, cache_folder: Optional[str] = None, use_auth_token: Union[bool, str, NoneType] = None)
 |  
 |  Loads or create a SentenceTransformer model, that can be used to map sentences / text to embeddings.
 |  
 |  :param model_name_or_path: If it is a filepath on disc, it loads the model from that path. If it is not a path, it first tries to download a pre-trained SentenceTransformer model. If that fails, tries to construct a model from Huggingface models repository with that name.
 |  :param modules: This parameter can be used to create custom SentenceTransformer models from scratch.
 |  :param device: Device (like 'cuda' / 'cpu') that should be used for computation. If None, chec

In [34]:
model_3.tokenizer.pad_token

'[PAD]'

In [35]:
model_3.tokenizer.pad_token_id

0

In [38]:
model_3.tokenizer.pad_token_type_id

0

In [45]:
model_3.tokenizer.__call__

<bound method PreTrainedTokenizerBase.__call__ of BertTokenizerFast(name_or_path='/Users/latchari/.cache/torch/sentence_transformers/sentence-transformers_multi-qa-MiniLM-L6-cos-v1/', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)>

In [37]:
help(model_3.tokenizer)

Help on BertTokenizerFast in module transformers.models.bert.tokenization_bert_fast object:

class BertTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  BertTokenizerFast(vocab_file=None, tokenizer_file=None, do_lower_case=True, unk_token='[UNK]', sep_token='[SEP]', pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', tokenize_chinese_chars=True, strip_accents=None, **kwargs)
 |  
 |  Construct a "fast" BERT tokenizer (backed by HuggingFace's *tokenizers* library). Based on WordPiece.
 |  
 |  This tokenizer inherits from [`PreTrainedTokenizerFast`] which contains most of the main methods. Users should
 |  refer to this superclass for more information regarding those methods.
 |  
 |  Args:
 |      vocab_file (`str`):
 |          File containing the vocabulary.
 |      do_lower_case (`bool`, *optional*, defaults to `True`):
 |          Whether or not to lowercase the input when tokenizing.
 |      unk_token (`str`, *optional*, defaults to `"[UNK]"`):

In [52]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id_3)

In [16]:
help(tokenizer)

Help on BertTokenizerFast in module transformers.models.bert.tokenization_bert_fast object:

class BertTokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  BertTokenizerFast(vocab_file=None, tokenizer_file=None, do_lower_case=True, unk_token='[UNK]', sep_token='[SEP]', pad_token='[PAD]', cls_token='[CLS]', mask_token='[MASK]', tokenize_chinese_chars=True, strip_accents=None, **kwargs)
 |  
 |  Construct a "fast" BERT tokenizer (backed by HuggingFace's *tokenizers* library). Based on WordPiece.
 |  
 |  This tokenizer inherits from [`PreTrainedTokenizerFast`] which contains most of the main methods. Users should
 |  refer to this superclass for more information regarding those methods.
 |  
 |  Args:
 |      vocab_file (`str`):
 |          File containing the vocabulary.
 |      do_lower_case (`bool`, *optional*, defaults to `True`):
 |          Whether or not to lowercase the input when tokenizing.
 |      unk_token (`str`, *optional*, defaults to `"[UNK]"`):

In [18]:
tokenizer.pretrained_vocab_files_map

{'vocab_file': {'bert-base-uncased': 'https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt',
  'bert-large-uncased': 'https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt',
  'bert-base-cased': 'https://huggingface.co/bert-base-cased/resolve/main/vocab.txt',
  'bert-large-cased': 'https://huggingface.co/bert-large-cased/resolve/main/vocab.txt',
  'bert-base-multilingual-uncased': 'https://huggingface.co/bert-base-multilingual-uncased/resolve/main/vocab.txt',
  'bert-base-multilingual-cased': 'https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt',
  'bert-base-chinese': 'https://huggingface.co/bert-base-chinese/resolve/main/vocab.txt',
  'bert-base-german-cased': 'https://huggingface.co/bert-base-german-cased/resolve/main/vocab.txt',
  'bert-large-uncased-whole-word-masking': 'https://huggingface.co/bert-large-uncased-whole-word-masking/resolve/main/vocab.txt',
  'bert-large-cased-whole-word-masking': 'https://huggingface.co/bert-large-cased-

In [20]:
tokenizer.pad_token

'[PAD]'

In [21]:
tokenizer.pad_token_id

0

In [30]:
tokenizer.pad

<bound method PreTrainedTokenizerBase.pad of BertTokenizerFast(name_or_path='sentence-transformers/paraphrase-MiniLM-L3-v2', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)>

In [ ]:
PaddingStrategy

In [47]:
from transformers import AutoConfig

In [48]:
config = AutoConfig.from_pretrained(model_id_3)

In [50]:
config.padding_strategy

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 config.padding_strategy                                                                      │
│   2                                                                                              │
│                                                                                                  │
│ /opt/homebrew/lib/python3.11/site-packages/transformers/configuration_utils.py:261 in            │
│ __getattribute__                                                                                 │
│                                                                                                  │
│   258 │   def __getattribute__(self, key):                                                       │
│   259 │   │   if key != "attribute_map" and key in super().__getattribute__("attribute_map"):    │
│   260 │   │   │   key = super().__getattribute__("attribute_map")[key]                           │
│ ❱ 261 │   │   return super().__getattribute__(key)                                               │
│   262 │                                                                                          │
│   263 │   def __init__(self, **kwargs):                                                          │
│   264 │   │   # Attributes with defaults                                                         │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'BertConfig' object has no attribute 'padding_strategy'

In [58]:
tokenizer.padding_side

'right'

In [59]:
model_3.tokenizer.padding_side

'right'

In [71]:
model_3.tokenizer.pad.strategy

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 model_3.tokenizer.pad.strategy                                                               │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'function' object has no attribute 'strategy'

In [77]:
help(tokenizer.pad)

Help on method pad in module transformers.tokenization_utils_base:

pad(encoded_inputs: Union[transformers.tokenization_utils_base.BatchEncoding, List[transformers.tokenization_utils_base.BatchEncoding], Dict[str, List[int]], Dict[str, List[List[int]]], List[Dict[str, List[int]]]], padding: Union[bool, str, transformers.utils.generic.PaddingStrategy] = True, max_length: Optional[int] = None, pad_to_multiple_of: Optional[int] = None, return_attention_mask: Optional[bool] = None, return_tensors: Union[str, transformers.utils.generic.TensorType, NoneType] = None, verbose: bool = True) -> transformers.tokenization_utils_base.BatchEncoding method of transformers.models.bert.tokenization_bert_fast.BertTokenizerFast instance
    Pad a single encoded input or a batch of encoded inputs up to predefined length or to the max sequence length
    in the batch.
    
    Padding side (left/right) padding token ids are defined at the tokenizer level (with `self.padding_side`,
    `self.pad_token_id` a